# Example sample-wise adaptation

In [19]:
import numpy as np
from numpy import unravel_index
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import ot
import scipy.io
import mne          
from mne.decoding import CSP
mne.set_log_level(verbose='warning') #to avoid info at terminal
import matplotlib.pyplot as pl
from random import seed
seed(30)
from MIOTDAfunctions import*

# get the functions from RPA package
import rpa.transfer_learning as TL

from pyriemann.classification import MDM
from pyriemann.estimation import Covariances
from pyriemann.utils.base import invsqrtm
import timeit

#ignore warning 
from warnings import simplefilter

# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)

## Methods definition

In [2]:
def SC(Gte, Yte, lda):
    
    start = timeit.default_timer()
    
    yt_predict = lda.predict(Gte)
    
    stop = timeit.default_timer()
    time = stop - start
    
    return yt_predict, time 

In [3]:
def SR(Data_S2, Labels_S2, re, Xtr, Ytr, Xte, Yte):
    
    start = timeit.default_timer()
    
    #Get Data
    Xtr2add = Data_S2[0:20*re+20]
    Ytr2add = Labels_S2[0:20*re+20]
    
    Xtr2 = np.vstack(((Xtr, Xtr2add)))
    Ytr2 = np.hstack(((Ytr, Ytr2add)))
        
    Ytr2 = Ytr2[len(Ytr2add):]
    Xtr2 = Xtr2[len(Ytr2add):]

    # Create a new CSP
    csp = CSP(n_components=6, reg='empirical', log=True, norm_trace=False, cov_est='epoch')
    
    #learn new csp filters
    Gtr = csp.fit_transform(Xtr2,Ytr2)
    
    #learn new lda
    lda = LinearDiscriminantAnalysis()
    lda.fit(Gtr,Ytr2)

    # Apply on new test data
    Gte = csp.transform(Xte)
    #ldatest
    yt_predict = lda.predict(Gte)
    
    # time
    stop = timeit.default_timer()
    time = stop - start
    
    return yt_predict, time 

In [4]:
def Sinkhorn_Transport(Gtr_daot, Ytr_daot, regu_, Gtr, Ytr, Gval, Gte, lda, metric, outerkfold, innerkfold, M):
    
    
    #time
    start = timeit.default_timer()
        
    otda = ot.da.SinkhornTransport(metric=metric, reg_e=regu_)
    #learn the map
    otda.fit(Xs=Gtr_daot, ys=Ytr_daot, Xt=Gval)
    
    #apply the mapping over source data
    transp_Xs = otda.transform(Xs=Gtr)

    # train a new classifier bases upon the transform source data
    lda.fit(transp_Xs, Ytr)
    
    # Compute acc
    yt_predict = lda.predict(Gte)
    
    # time
    stop = timeit.default_timer()
    time = stop - start  
    
    return yt_predict, time

In [5]:
def GroupLasso_Transport(Gtr_daot, Ytr_daot, regu_, Gtr, Ytr, Gval, Gte, lda, metric, outerkfold, innerkfold, M):
        
    #time
    start = timeit.default_timer()
    
        
    otda = ot.da.SinkhornL1l2Transport(metric = metric ,reg_e = regu_[0], reg_cl = regu_[1])
    otda.fit(Xs=Gtr_daot, ys=Ytr_daot, Xt=Gval)

    #transport taget samples onto source samples
    transp_Xs = otda.transform(Xs=Gtr)

    # train a new classifier bases upon the transform source data
    lda.fit(transp_Xs, Ytr)

    # Compute acc
    yt_predict = lda.predict(Gte)   
    # time
    stop = timeit.default_timer()
    time = stop - start 
        
    
    return yt_predict, time

In [6]:
def Backward_Sinkhorn_Transport(Gtr_daot, Ytr_daot, regu_, Gtr, Ytr, Gval, Gte, lda, metric, outerkfold, innerkfold, M):
   
    # time
    start = timeit.default_timer()
      
    # Transport plan
    botda = ot.da.SinkhornTransport(metric=metric, reg_e=regu_)
    botda.fit(Xs=Gval, ys=Yval, Xt=Gtr_daot)
    
    #transport testing samples
    transp_Xt_backward = botda.transform(Xs=Gte)
    
    # Compute accuracy without retraining    
    yt_predict = lda.predict(transp_Xt_backward)
    
    # time
    stop = timeit.default_timer()
    time = stop - start
    
    return yt_predict, time

In [7]:
def Backward_GroupLasso_Transport(Gtr_daot, Ytr_daot, regu_, Gtr, Ytr, Gval, Gte, lda, metric, outerkfold, innerkfold, M):
       
    #time
    start = timeit.default_timer()
      
    botda = ot.da.SinkhornL1l2Transport(metric=metric, reg_e=regu_[0], reg_cl=regu_[1])
    botda.fit(Xs=Gval, ys=Yval, Xt=Gtr_daot)
    
    #transport testing samples
    transp_Xt_backward=botda.transform(Xs=Gte)
    
    # Compute accuracy without retraining    
    yt_predict = lda.predict(transp_Xt_backward)
    
    # time
    stop = timeit.default_timer()
    time = stop - start
    
    
    return yt_predict, time

In [8]:
def RPA(Xtr, Xval, Xte, Ytr, Yval, Yte):
    
    # time
    start = timeit.default_timer()
    # cov matrix estimation
    cov_tr = Covariances().transform(Xtr)
    cov_val= Covariances().transform(Xval)
    cov_te = Covariances().transform(Xte)
        
    clf = MDM()
    source={'covs':cov_tr, 'labels': Ytr}
    target_org_train={'covs':cov_val, 'labels': Yval}
    target_org_test={'covs':cov_te, 'labels': Yte}
    
    # re-centered matrices
    source_rct, target_rct_train, target_rct_test = TL.RPA_recenter(source, target_org_train, target_org_test)   
    # stretched the re-centered matrices
    source_rcs, target_rcs_train, target_rcs_test = TL.RPA_stretch(source_rct, target_rct_train, target_rct_test)
    # rotate the re-centered-stretched matrices using information from classes
    source_rpa, target_rpa_train, target_rpa_test = TL.RPA_rotate(source_rcs, target_rcs_train, target_rcs_test)
    
    # get data
    covs_source, y_source = source_rpa['covs'], source_rpa['labels']
    covs_target_train, y_target_train = target_rpa_train['covs'], target_rpa_train['labels']
    covs_target_test, y_target_test = target_rpa_test['covs'], target_rpa_test['labels']
    
    # append train and validation data
    covs_train = np.concatenate([covs_source, covs_target_train])
    y_train = np.concatenate([y_source, y_target_train])
    
    # train
    clf.fit(covs_train, y_train)
    
    # test
    covs_test = covs_target_test
    y_test = y_target_test
    yt_predict = clf.predict(covs_test)
    
    # time
    stop = timeit.default_timer()
    time = stop - start
    
    return yt_predict, time

In [9]:
def EU(Xtr,Xval,Xte,Ytr,Yval,Yte):
    
    # time
    start = timeit.default_timer()
    # Estimate single trial covariance
    cov_tr = Covariances().transform(Xtr)
    cov_val= Covariances().transform(Xval)
    
    Ctr = cov_tr.mean(0)
    Cval = cov_val.mean(0)
    
    # aligment
    Xtr_eu = np.asarray([np.dot(invsqrtm(Ctr), epoch) for epoch in Xtr])
    Xval_eu = np.asarray([np.dot(invsqrtm(Cval), epoch) for epoch in Xval])
    Xte_eu = np.asarray([np.dot(invsqrtm(Cval), epoch) for epoch in Xte])

    # append train and validation data
    x_train = np.concatenate([Xtr_eu, Xval_eu])
    y_train = np.concatenate([Ytr, Yval])

    # train new csp+lda
    csp = CSP(n_components=6, reg='empirical', log=True, norm_trace=False, cov_est='epoch')
    # learn csp filters
    Gtr = csp.fit_transform(x_train,y_train)
    
    # learn lda
    lda = LinearDiscriminantAnalysis()
    lda.fit(Gtr,y_train)
    
    # test
    Gte = csp.transform(Xte_eu)  
    # acc
    yt_predict = lda.predict(Gte)
    
    # time
    stop = timeit.default_timer()
    time = stop - start
        
    return yt_predict, time

### OTDA params

In [10]:
rango_cl=[0.1, 1, 10]
rango_e=[0.1, 1, 10] 
metric = 'sqeuclidean'
outerkfold = 20
innerkfold = None
M=20

### Load and filter data

In [11]:
fName = 'Data/DataSession1_S9.mat'
s = scipy.io.loadmat(fName)

Data_S1=s["X"]
Labels_S1=s["y"]
Labels_S1=np.squeeze(Labels_S1)

#filterting with mne
[nt, nc, ns]=np.shape(Data_S1)
Data_S1=np.reshape(Data_S1, [nt, nc*ns])
Data_S1=mne.filter.filter_data(Data_S1, 128, 8, 30)
Data_S1=np.reshape(Data_S1, [nt,nc,ns])

fName = 'Data/DataSession2_S9.mat'
s2 = scipy.io.loadmat(fName)

Data_S2=s2["X"]
Labels_S2=s2["y"]
Labels_S2=np.squeeze(Labels_S2)

#filterting with mne
[nt, nc, ns]=np.shape(Data_S2)
Data_S2=np.reshape(Data_S2, [nt, nc*ns])
Data_S2=mne.filter.filter_data(Data_S2, 128, 8, 30)
Data_S2=np.reshape(Data_S2, [nt,nc,ns])

In [12]:
# define variables to save the predictions
yt_predict_sc=[]
yt_predict_sr=[]
yt_predict_1=[]
yt_predict_2=[]
yt_predict_3=[]
yt_predict_4=[]
yt_predict_rpa=[]
yt_predict_eu=[]

### Learn CSP+LDA from source data (Data_S1)

In [13]:
Xtr = Data_S1
Ytr = Labels_S1
csp = CSP(n_components=6, reg='empirical', log=True, norm_trace=False, cov_est='epoch')
#learn csp filters
Gtr = csp.fit_transform(Xtr, Ytr)
#learn lda
lda = LinearDiscriminantAnalysis()
lda.fit(Gtr,Ytr)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)

### Use the first 20 trials as the validation set

In [14]:
Labels_te=Labels_S2[20:]
##
Xval=Data_S2[0:20]
Yval=Labels_S2[0:20]
##
Gval=csp.transform(Xval)

### select source subset

In [15]:
#for fotda, create a new classifier (clf)
clf=LinearDiscriminantAnalysis()
G_FOTDAs_, Y_FOTDAs_, regu_FOTDAs_=\
SelectSubsetTraining_OTDAs(Gtr, Ytr, Gval, Yval, rango_e, clf, metric, outerkfold, innerkfold, M)
G_FOTDAl1l2_, Y_FOTDAl1l2_, regu_FOTDAl1l2_=\
    SelectSubsetTraining_OTDAl1l2(Gtr, Ytr, Gval, Yval, rango_e, rango_cl, clf, metric, outerkfold, innerkfold, M)
#for botda, use the already trained classifier (lda)
G_BOTDAs_, Y_BOTDAs_, regu_BOTDAs_=\
SelectSubsetTraining_BOTDAs(Gtr, Ytr, Gval, Yval, rango_e, lda, metric, outerkfold, innerkfold, M)
G_BOTDAl1l2_, Y_BOTDAl1l2_, regu_BOTDAl1l2_=\
SelectSubsetTraining_BOTDAl1l2(Gtr, Ytr, Gval, Yval, rango_e, rango_cl, lda, metric, outerkfold, innerkfold, M)

### For each sample, make the data adaptation

In [16]:
for re in range(1,len(Labels_te)+1):
    if np.mod(re,10)==0 : print('Running testing trial={:1.0f}'.format(re))
    #testing trial
    Xte=Data_S2[20+(re-1):20+(re)]
    Xte=Xte.reshape(1, nc, ns)
    Yte=Labels_S2[20+(re-1):20+(re)]
    
    Xval=np.vstack((Xval, Xte))
    Yval=np.hstack((Yval, Yte))
    
    #csp estimation
    Gval=csp.transform(Xval)
    Gte=csp.transform(Xte)
         
    # SC  
    yt_predict, time_sc = SC(Gte, Yte, lda)
    yt_predict_sc.append(yt_predict)

    
    # SR
    yt_predict, time_sr = SR(Data_S2, Labels_S2, re, Xtr, Ytr, Xte, Yte)
    yt_predict_sr.append(yt_predict)

    #%% # Sinkhorn Transport
    yt_predict, time_fs = Sinkhorn_Transport(G_FOTDAs_, Y_FOTDAs_, regu_FOTDAs_, Gtr, Ytr, Gval, Gte, clf, metric, outerkfold, innerkfold, M)
    yt_predict_1.append(yt_predict)

    #%% # Group-Lasso Transport
    yt_predict, time_fg = GroupLasso_Transport(G_FOTDAl1l2_, Y_FOTDAl1l2_, regu_FOTDAl1l2_, Gtr, Ytr, Gval, Gte, clf, metric, outerkfold, innerkfold, M)
    yt_predict_2.append(yt_predict)

    #%% # Backward Sinkhorn Transport
    yt_predict, time_bs = Backward_Sinkhorn_Transport(G_BOTDAs_, Y_BOTDAs_, regu_BOTDAs_, Gtr, Ytr, Gval, Gte, lda, metric, outerkfold, innerkfold, M)
    yt_predict_3.append(yt_predict)

    #%% # Backward Group-Lasso Transport
    yt_predict, time_bg = Backward_GroupLasso_Transport(G_BOTDAl1l2_, Y_BOTDAl1l2_, regu_BOTDAl1l2_, Gtr, Ytr, Gval, Gte, lda, metric, outerkfold, innerkfold, M)
    yt_predict_4.append(yt_predict)

    # Riemann
    yt_predict, time_rpa = RPA(Xtr,Xval,Xte,Ytr,Yval,Yte)
    yt_predict_rpa.append(yt_predict)

    # Euclidean
    yt_predict, time_eu = EU(Xtr,Xval,Xte,Ytr,Yval,Yte)
    yt_predict_eu.append(yt_predict)
    
    #save times
    times = [time_sr, time_rpa, time_eu, time_fs, time_fg, time_bs, time_bg]
        
    if re==1:
        times_se = times
    else:
        times_se = np.vstack((times_se, times))  

Running testing trial=10
Running testing trial=20
Running testing trial=30
Running testing trial=40
Running testing trial=50
Running testing trial=60
Running testing trial=70
Running testing trial=80
Running testing trial=90
Running testing trial=100
Running testing trial=110
Running testing trial=120
Running testing trial=130
Running testing trial=140


In [18]:
# compute accuracy 
yt_predict_4=np.squeeze(np.asarray(yt_predict_4))
yt_predict_3=np.squeeze(np.asarray(yt_predict_3))
yt_predict_2=np.squeeze(np.asarray(yt_predict_2))
yt_predict_1=np.squeeze(np.asarray(yt_predict_1))
yt_predict_sc=np.squeeze(np.asarray(yt_predict_sc))
yt_predict_sr=np.squeeze(np.asarray(yt_predict_sr))

acc_botdal1l2=accuracy_score(Labels_te, yt_predict_4)
acc_botdas=accuracy_score(Labels_te, yt_predict_3)
acc_fotdal1l2=accuracy_score(Labels_te, yt_predict_2)
acc_fotdas=accuracy_score(Labels_te, yt_predict_1)
acc_sc=accuracy_score(Labels_te, yt_predict_sc)
acc_sr=accuracy_score(Labels_te, yt_predict_sr)

#print accuracy
acc={}
acc["sc"]=acc_sc
acc["sr"]=acc_sr
acc["fotda_s"]=acc_fotdas
acc["fotda_l1l2"]=acc_fotdal1l2
acc["botda_s"]=acc_botdas
acc["botda_l1l2"]=acc_botdal1l2
    

#print computing time
mean_time = np.mean(times_se, axis=0)
time = {}
time["sr"] = round(mean_time[0],3)
time["rpa"] = round(mean_time[1],3)
time["eu"] = round(mean_time[2],3)
time["fotda_s"] = round(mean_time[3],3)
time["fotda_l1l2"] = round(mean_time[4],3)
time["botda_s"] = round(mean_time[5],3)
time["botda_l1l2"] = round(mean_time[6],3)
    
print('ACC')
print(acc)
print('CT')
print(time)

ACC
{'sc': 0.7214285714285714, 'sr': 0.8571428571428571, 'fotda_s': 0.7857142857142857, 'fotda_l1l2': 0.7642857142857142, 'botda_s': 0.7928571428571428, 'botda_l1l2': 0.9857142857142858}
CT
{'sr': 1.159, 'rpa': 4.12, 'eu': 1.988, 'fotda_s': 0.013, 'fotda_l1l2': 0.29, 'botda_s': 0.012, 'botda_l1l2': 0.123}
